[View in Colaboratory](https://colab.research.google.com/github/ameasure/try_git/blob/master/FastAI.ipynb)

In [1]:
!pip install fastai

    100% |████████████████████████████████| 92kB 2.8MB/s 
    100% |████████████████████████████████| 61kB 7.0MB/s 
    100% |████████████████████████████████| 519.5MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5a11e000 @  0x7f203a03d1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 112kB 10.9MB/s 
    100% |████████████████████████████████| 112kB 20.9MB/s 
    100% |████████████████████████████████| 2.2MB 1.1MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.15 which is incompatible.


In [2]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 61.2MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
import pandas as pd


df = pd.read_csv(IMDB_PATH/'train.csv', header=None)
df.head()